In [5]:
import tensorflow as tf
import numpy as np
import warnings
import random
import os
import sys
sys.path.append('../src')
from models.unet import UNet
from tensorflow.python.saved_model import tag_constants

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

In [20]:
IMG_SIZE = 101
IMG_CHANNELS = 1
path_train = '../data/train'
path_test = '../data/test'

train_ids = next(os.walk(path_train + "/images"))[2]
test_ids = next(os.walk(path_test + "/images"))[2]

train_images = np.zeros((len(X_train), IMG_SIZE, IMG_SIZE, IMG_CHANNELS), dtype=np.uint8)
train_labels = np.zeros((len(X_train), IMG_SIZE, IMG_SIZE, 1), dtype=np.bool)

print('Getting and resizing train images and masks without padding ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(X_train), total=len(X_train)):
    img = imread(path_train + "/images/" + id_)[:,:,:IMG_CHANNELS]
    train_images[n] = img

    mask = imread(path_train + "/masks/" + id_)
    mask = np.expand_dims(mask, axis = -1)

    train_labels[2] = mask


test_images = np.zeros((len(test_ids), IMG_SIZE, IMG_SIZE, IMG_CHANNELS), dtype=np.uint8)
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    img = imread(path_test + "/images/" + id_)[:,:,:IMG_CHANNELS]
    test_images[n] = img

Getting and resizing train images and masks without padding ... 


100%|██████████| 18000/18000 [00:05<00:00, 3075.49it/s]


In [94]:
val_data = tf.data.Dataset.from_tensor_slices(np.arange(25))

In [96]:
val_data = val_data.batch(10)
val_data = val_data.shuffle(2)

In [91]:
iterator = val_data.make_one_shot_iterator()
next_element = iterator.get_next()

In [98]:
with tf.Session() as sess:
    for _ in range(5):
        val_data = val_data.shuffle(2)
        iterator = val_data.make_one_shot_iterator()
        next_element = iterator.get_next()
        print(sess.run(next_element))

InvalidArgumentError: Cannot batch tensors with different shapes in component 0. First element had shape [10] and element 2 had shape [5].
	 [[Node: IteratorGetNext_35 = IteratorGetNext[output_shapes=[[?,?]], output_types=[DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_35)]]

Caused by op 'IteratorGetNext_35', defined at:
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chasm/.virtualenvs/saltid/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chasm/.virtualenvs/saltid/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/chasm/.virtualenvs/saltid/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "

In [8]:
sess = tf.InteractiveSession()

In [9]:
saver = tf.train.import_meta_graph('../models/loss-0.4969549126923084-39-39.meta')

In [10]:
graph = tf.get_default_graph()

In [12]:
logits = graph.get_tensor_by_name('logits/BiasAdd:0')

In [13]:
finalimg = tf.nn.sigmoid(logits)

In [26]:
i = 0
batch_X, batch_Y = next_train_batch(50, i, i == 0)


NameError: name 'images' is not defined

In [ ]:
output = sess.run([finalimg], feed_dict={X:batch_X})